In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from skopt.space import Real, Integer
from skopt.utils import use_named_args
from skopt import gp_minimize
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import LassoCV
import xgboost as xgb
import lightgbm as lgb
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras import backend as K
from sklearn.ensemble import RandomForestRegressor

Using TensorFlow backend.


In [2]:
data = pd.read_pickle('data/model/model3_w5.pkl')
target = pd.read_csv('data/target_competencia_ids.csv')
target_obj = pd.read_csv('data/target_competencia_ids.csv')

In [3]:
target['ref_hash'] = target['ref_hash'].apply(lambda x: int(str(x)[:-3]))
target.drop_duplicates('ref_hash', inplace=True)

In [4]:
features = {'n_auctions':0, 'diff_auctions': 'mean', 'mean_time_auction': 'mean', 'first_auction_sec': 'mean',
            'last_auction_sec': 'mean', 'ref_type_id_1': 0, 'ref_type_id_7': 0, 'source_id_0': 0, 'source_id_1': 0, 'source_id_2':0,
            'source_id_3':0, 'source_id_4':0, 'source_id_5':0, 'source_id_6':0,
            'source_id_7':0, 'source_id_8':0, 'source_id_9':0, 'mean_day': 'mean', 'day_0': 0,'day_1': 0, 'day_2': 0,
            'n_events': 0, 'mean_time_events': 'mean', 'wifi_events_mean': 'mean', 'diff_events': 0, 'first_event_sec': 0,
           'last_event_sec': 0, 'attributed_events_mean': 0, 'touchX_mean': 'mean', 'touchY_mean': 'mean', 'timeToClick_mean': 'mean',
           'latitude_mean': 'mean', 'longitude_mean': 'mean', 'n_clicks': 0, 'mean_time_install': 0, 'timeToClick_mean': 'mean',
           'first_install_sec': 0, 'last_install_sec': 0, 'n_installs': 0, 'mean_time_install': 'mean', 'attributed_installs_mean': 'mean',
           'wifi_installs_mean': 'mean', 'diff_installs': 'mean', 'mean_time_install': 'mean', 'wifi_connection': 'mean', 'first_click_sec': 'mean',
           'last_click_sec': 'mean', 'diff_clicks': 'mean', 'mean_time_click': 'mean'}

for feature, method in features.items():
    if feature not in data:
        continue
    if method == 0:
        data[feature] = data[feature].fillna(0)
    elif method == 'mean':
        data[feature] = data[feature].fillna(data[feature].mean())
    elif method == 'max':
        data[feature] = data[feature].fillna(data[feature].max())
    elif method == 'min':
        data[feature] = data[feature].fillna(data[feature].min())
    elif method == -1:
        data[feature] = data[feature].fillna(-1)

In [5]:
pd.merge(data, target, on='ref_hash', how='inner').shape

(4037, 48)

In [153]:
data.shape

(580977, 47)

In [154]:
target.shape

(4037, 2)

In [7]:
data = pd.merge(data, target, how='inner', on='ref_hash')

In [8]:
predictions_auctions = data[['ref_hash']]
predictions_installs = data[['ref_hash']]
predictions_auctions['ref_hash'] = predictions_auctions['ref_hash'].apply(lambda x: str(int(x)) + '_st')
predictions_installs['ref_hash'] = predictions_installs['ref_hash'].apply(lambda x: str(int(x)) + '_sc')
data.drop(columns=['ref_hash'], inplace=True)
model_auctions = lgb.Booster(model_file='modelos/lightgbm_auctions')
model_installs = lgb.Booster(model_file='modelos/lightgbm_installs')
predictions_auctions['obj'] = model_auctions.predict(data)
predictions_installs['obj'] = model_installs.predict(data)
predictions = pd.concat([predictions_auctions, predictions_installs])

D:\Martin\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Martin\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
D:\Martin\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [9]:
pd.merge(predictions, target_obj, on='ref_hash', how='inner').shape

(8074, 3)

In [10]:
predictions.to_csv('predicciones/predicciones1.csv')